In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime

from sklearn import tree
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn import preprocessing
from sklearn.externals import joblib
from sklearn.ensemble import RandomForestClassifier
import lightgbm as lgb
import gc

%matplotlib inline

In [3]:
plt.rc('figure',dpi=100);

In [3]:
avisos = pd.read_csv("../../Data/fiuba_entrenamiento/gian/avisos.csv")
postulaciones = pd.read_csv("../../Data/fiuba_entrenamiento/gian/postulaciones.csv")
postulantes = pd.read_csv("../../Data/fiuba_entrenamiento/gian/postulantes.csv")

postulantes['sexo']=postulantes['sexo'].astype('category')
postulantes['Doctorado']=postulantes['Doctorado'].astype('category')
postulantes['Master']=postulantes['Master'].astype('category')
postulantes['Otro']=postulantes['Otro'].astype('category')
postulantes['Posgrado']=postulantes['Posgrado'].astype('category')
postulantes['Secundario']=postulantes['Secundario'].astype('category')
postulantes['Terciario/Técnico']=postulantes['Terciario/Técnico'].astype('category')
postulantes['Universitario']=postulantes['Universitario'].astype('category')

avisos['nombre_zona']=avisos['nombre_zona'].astype('category')
avisos['tipo_de_trabajo']=avisos['tipo_de_trabajo'].astype('category')
avisos['nivel_laboral']=avisos['nivel_laboral'].astype('category')
avisos['nombre_area']=avisos['nombre_area'].astype('category')
avisos['denominacion_empresa']=avisos['denominacion_empresa'].astype('category')

In [4]:
vistas = pd.read_csv("../../Data/fiuba_entrenamiento/gian/vistas.csv")
vistas.head()

,idAviso,timestamp,idpostulante,dia,mes,semana,diadelasemana,hora
0,1111780242,2018-02-23 18:38:13.187,YjVJQ6Z,23,2,8,Viernes,18
1,1112263876,2018-02-23 18:38:14.296,BmVpYoR,23,2,8,Viernes,18
2,1112327963,2018-02-23 18:38:14.329,wVkBzZd,23,2,8,Viernes,18
3,1112318643,2018-02-23 18:38:17.921,OqmP9pv,23,2,8,Viernes,18
4,1111903673,2018-02-23 18:38:18.973,DrpbXDP,23,2,8,Viernes,18


In [5]:
vistas.drop(['timestamp','dia','mes','semana','diadelasemana','hora'],axis=1,inplace=True)
vistas['cantidad']=0
vistas2=vistas.groupby(['idAviso','idpostulante'],as_index=False).agg({'cantidad':'count'})
vistas2=vistas2.rename(columns={'idAviso': 'idaviso'})
vistas2.head()

,idaviso,idpostulante,cantidad
0,18,BolNL,2
1,48375,RwVdKR,1
2,169730,1KjXB,2
3,169730,2AKzxa,2
4,169730,6LJ64,1


In [6]:
vistas=vistas.groupby(['idpostulante'],as_index=False).agg({'cantidad':'count'});
vistas=vistas.rename(columns={'idAviso': 'idaviso'})
vistas.head()

,idpostulante,cantidad
0,0002q,15
1,0005E,58
2,000R8,14
3,001XE,26
4,003k9,32


In [7]:
vistas2=pd.merge(vistas2,vistas,on='idpostulante',how='left')

In [8]:
vistas2.head()

,idaviso,idpostulante,cantidad_x,cantidad_y
0,18,BolNL,2,28
1,48375,RwVdKR,1,15
2,169730,1KjXB,2,99
3,169730,2AKzxa,2,2
4,169730,6LJ64,1,23


In [9]:
vistas2['cantidad']=vistas2['cantidad_x']/vistas2['cantidad_y']
vistas2.drop(['cantidad_x','cantidad_y'],axis=1,inplace=True)
vistas=vistas2
vistas2.head()

,idaviso,idpostulante,cantidad
0,18,BolNL,0.071429
1,48375,RwVdKR,0.066667
2,169730,1KjXB,0.020202
3,169730,2AKzxa,1.000000
4,169730,6LJ64,0.043478


In [10]:
prediccion = pd.read_csv("../../Data/fiuba_entrenamiento/test_final_100k.csv")
print(len(prediccion))
prediccion=pd.merge(prediccion,postulantes,on='idpostulante',how='inner')
print(len(prediccion))
prediccion=pd.merge(prediccion,avisos,on='idaviso',how='inner')
print(len(prediccion))
prediccion=pd.merge(prediccion,vistas,on=['idaviso','idpostulante'],how='left')
prediccion.cantidad=prediccion.cantidad.fillna(0)
prediccion.sample(10)

100000
100000
100000


,id,idaviso,idpostulante,sexo,edad,Doctorado,Master,Otro,Posgrado,Secundario,...,viajar,empresa_importante,capacitacion,remuneracion_pretendida,graduados,estudiantes,conocimientos_especificos,buen_ambiente,discriminacion,cantidad
91080,95467,1112489558,aXw0A0,MASC,31.0,-,-,-,-,-,...,False,False,False,False,False,False,False,True,False,0.000000
4435,65272,1112458667,1Q4L0oB,FEM,23.0,-,-,Graduado,-,Graduado,...,False,False,False,False,False,False,False,False,False,0.000000
88273,97550,1112501658,5xOr3v,FEM,34.0,-,-,-,-,Graduado,...,False,False,True,False,False,False,False,False,False,0.000000
14129,61703,1112455597,W9WbWWl,MASC,20.0,-,-,-,-,-,...,False,True,False,False,False,False,False,True,False,0.007299
28019,25229,1112334784,8MPWwqx,FEM,30.0,-,-,-,-,Graduado,...,False,False,True,False,False,False,False,False,False,0.030000
12060,80036,1112464787,eOk5D4,MASC,28.0,-,-,-,-,-,...,False,False,False,False,False,True,False,False,False,0.000000
82930,31394,1112386543,ER2vZz,MASC,29.0,-,-,-,-,-,...,False,False,False,False,False,False,False,False,False,0.000000
88583,45842,1112429125,5QKNav,MASC,32.0,-,-,-,-,-,...,False,True,False,True,False,False,False,False,False,0.000000
39497,31806,1112389478,wVYN9RZ,MASC,22.0,-,-,-,-,Graduado,...,False,False,False,True,False,False,False,False,False,0.000000
79006,26448,1112341154,ZxeEQZ,FEM,28.0,-,-,Graduado,-,Graduado,...,False,False,False,False,False,False,False,False,False,0.000000


In [11]:
prediccion.sexo.value_counts()

MASC          49198
FEM           48022
NO_DECLARA     2780
0.0               0
Name: sexo, dtype: int64

In [12]:
prediccion.edad.value_counts()

29.0     4728
31.0     4486
32.0     4349
30.0     4205
34.0     4129
27.0     4108
33.0     4075
28.0     4025
35.0     3960
24.0     3615
37.0     3612
36.0     3568
26.0     3547
25.0     3516
38.0     3469
39.0     3161
23.0     3044
200.0    2959
40.0     2762
22.0     2574
41.0     2479
42.0     2333
21.0     2260
43.0     2178
20.0     1798
44.0     1653
19.0     1543
45.0     1379
46.0     1287
47.0     1163
         ... 
62.0      132
63.0       96
65.0       62
64.0       60
66.0       50
67.0       36
69.0       30
68.0       24
17.0       17
72.0       15
71.0       11
70.0        9
16.0        7
74.0        6
73.0        5
78.0        3
76.0        3
12.0        2
75.0        2
79.0        2
102.0       2
14.0        1
11.0        1
77.0        1
10.0        1
103.0       1
13.0        1
9.0         1
81.0        1
15.0        1
Name: edad, Length: 75, dtype: int64

In [14]:
prediccion.cantidad.mean()

0.01881988630191256

In [17]:
prediccion['nombre_area'].value_counts()

Ventas                               13238
Administración                        8514
Comercial                             7879
Producción                            7532
Atención al Cliente                   4100
Call Center                           3209
Recepcionista                         2425
Contabilidad                          2218
Almacén / Depósito / Expedición       1944
Tesorería                             1919
Telemarketing                         1690
Mantenimiento y Limpieza              1587
Programación                          1570
Tecnologia / Sistemas                 1556
Logística                             1491
Recursos Humanos                      1386
Marketing                             1312
Soporte Técnico                       1266
Mantenimiento                         1228
Oficios y Profesiones                 1179
Gastronomia                           1146
Seguridad                             1099
Legal                                 1073
Administrac